In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image, ImageFile
import openai
from fpdf import FPDF

# Allow loading corrupted images (optional safety)
ImageFile.LOAD_TRUNCATED_IMAGES = True

# ========= 1. Load Classifier Model =========
class FractureClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.fc(self.conv(x))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FractureClassifier().to(device)
model.load_state_dict(torch.load("fracture_classifier.pth", map_location=device))
model.eval()

# ========= 2. Image Preprocessing =========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def predict_fracture(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        return "fractured" if predicted.item() == 0 else "normal"

# ========= 3. Generate Report using GPT-4 =========
openai.api_key = "your-api-key-here"  # ⛔ Replace this or load from env

def generate_report(prediction: str):
    prompt = (
        f"You are a medical radiologist. Generate a detailed and professional report "
        f"for a hand X-ray image. The model has classified the image as '{prediction}'. "
        f"Include clinical terminology and structure the report with Findings, Impression, and Recommendations."
    )

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
        max_tokens=300
    )

    return response["choices"][0]["message"]["content"]

# ========= 4. Save Report to PDF =========
def save_pdf(report: str, filename="xray_report.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    for line in report.split("\n"):
        pdf.multi_cell(0, 10, line)

    pdf.output(filename)
    print(f"✅ Report saved to: {filename}")

# ========= 5. Run Everything =========
if __name__ == "__main__":
    image_path = "your_xray_image.jpg"  # 🔁 Replace this path
    prediction = predict_fracture(image_path)
    report = generate_report(prediction)
    save_pdf(report)
